In [11]:
# Included libraries
import numpy as np
import pandas as pd
import talib as ta

In [12]:
# Load CSV
df = pd.read_csv("../data/MarketData.csv", header=[3,4,5,6])
df.columns = df.columns.get_level_values(1) # Truncate to 1 header
df.rename(columns={df.columns[0]:"Date"}, inplace=True) # This is a bit inefficient as this name exists in level 0...
df.rename(columns={'LMCADS03 Comdty':'Copper',
                   'LMAHDS03 Comdty':'Aluminium',
                   'LMZSDS03 Comdty':'Zinc',
                   'LMPBDS03 Comdty':'Lead',
                   'LMSNDS03 Comdty':'Tin',
                   'CL1 Comdty':'CL1'},
                    inplace=True)
df.head(5)

,Date,Copper,Aluminium,Zinc,Lead,Tin,CL1
0,01/01/2010,7375.0,2230.0,2560.0,2432.0,16950,79.36
1,04/01/2010,7500.0,2267.0,2574.0,2515.0,17450,81.51
2,05/01/2010,7485.0,2302.0,2575.0,2522.5,17375,81.77
3,06/01/2010,7660.0,2377.0,2718.0,2680.0,17825,83.18
4,07/01/2010,7535.0,2310.0,2607.0,2599.0,17475,82.66


In [13]:
# Filter all data except 2021 Zinc and Copper
df = df[['Date', 'Copper', 'Zinc']]
df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y") # Convert to pd datetime friendly format
df = df[(df['Date'] > '2019-12-31') & (df['Date'] < '2022-01-01')]
df = df.reset_index(drop=True) # Reset index after truncation
df.tail(5)

,Date,Copper,Zinc
518,2021-12-27,9568.0,3519.0
519,2021-12-28,9568.0,3519.0
520,2021-12-29,9680.5,3513.0
521,2021-12-30,9691.5,3532.5
522,2021-12-31,9720.5,3534.0


In [14]:
# Function to deal with TA calculations
def ta_functions(input):
   input['RSI_Copper'] = ta.RSI(input['Copper']) # RSI Copper

   # MACD Copper
   input['MACD_Copper'], input['MACD_sig_Copper'], input['MACD_hist_Copper'] = ta.MACD(input['Copper'], fastperiod=12, slowperiod=26, signalperiod=9)

   input['RSI_Zinc'] = ta.RSI(input['Zinc']) # RSI Zinc

   # MACD Zinc
   input['MACD_Zinc'], input['MACD_sig_Zinc'], input['MACD_hist_Zinc'] = ta.MACD(input['Zinc'], fastperiod=12, slowperiod=26, signalperiod=9)

   return input

In [15]:
# Pass dataframe through TA calculations and add columns
df = ta_functions(df)
df.tail(5)

,Date,Copper,Zinc,RSI_Copper,MACD_Copper,MACD_sig_Copper,MACD_hist_Copper,RSI_Zinc,MACD_Zinc,MACD_sig_Zinc,MACD_hist_Zinc
518,2021-12-27,9568.0,3519.0,51.692406,-2.500190,-20.259043,17.758853,66.260897,66.849430,41.187284,25.662145
519,2021-12-28,9568.0,3519.0,51.692406,0.815513,-16.044131,16.859645,66.260897,69.361656,46.822159,22.539497
520,2021-12-29,9680.5,3513.0,56.618811,12.378352,-10.359635,22.737987,65.418907,70.060845,51.469896,18.590949
521,2021-12-30,9691.5,3532.5,57.079706,22.173985,-3.852911,26.026896,66.891416,71.365785,55.449074,15.916711
522,2021-12-31,9720.5,3534.0,58.336450,31.909321,3.299536,28.609786,67.007795,71.694546,58.698168,12.996378


In [16]:
# The following blocks were imported from question 2,
# in order to populate an SQL table with async inserts.
# This time a logging mechanism is also added based on the session context manager

# Included libraries
from __future__ import annotations
from asyncio import current_task, run
from contextlib import asynccontextmanager
from collections.abc import AsyncGenerator
import logging

# SQL
from sqlalchemy import Column, MetaData, select, String, Table, event, text, create_engine, insert, update, delete
from sqlalchemy.engine import Engine
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession, async_scoped_session
from sqlalchemy.orm import sessionmaker

In [17]:
engine = create_async_engine('sqlite+aiosqlite://', echo=True) # SQL engine
meta = MetaData() # Metadata for SQL session - used to generate table object later
maker = sessionmaker(engine, class_=AsyncSession)
scope = async_scoped_session(maker, current_task)
tablename = 'MarketData'

In [18]:
# Generate sessions via contextmanager for asynchronous function calls.
# The point of this is to allow multithreading/concurrency of DB I/O operations
@asynccontextmanager
async def get_session():
   async with scope() as session:
      yield session # Yield instead of return to pick up where it left off on every function call

In [19]:
# Not a 100% about this, I think in the output for the cell below with echo=True on the engine it shows
# multiple instances of async inserts being called, however I am not certain as to why the logging message
# does not show.

# Decorator that wraps the async insert function with a logging routine
@asynccontextmanager
async def logger(f):
   def wrap(*arguments, **kwarguments): # Wrapper takes in more than 1 arg
      logging.log(msg="IO start", level=logging.DEBUG)
      f(*arguments, **kwarguments)
   return wrap # Return wrapped function

In [20]:
# Asynchronous SQL CRUD operations
async def main():

   # Obtain a session variable for each instance
   async with get_session() as session:

      conn = await session.connection()

      # CSV to SQL
      await conn.run_sync(
         lambda sync_conn: df.to_sql(tablename, con=sync_conn)
      )

      # Define SQL table object
      marketdata = await conn.run_sync(
         lambda conn: Table(tablename, meta, autoload_with=conn, sqlite_autoincrement=True)
      )

      # Read table
      async with engine.connect() as conn:
         result = await conn.execute(select(marketdata))
         print(result.fetchall())

      # Permanently store changes to database
      await session.commit()

# Launch asynchronous task
await main()

2024-02-18 19:40:46,715 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-18 19:40:46,722 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("MarketData")
2024-02-18 19:40:46,724 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-18 19:40:46,728 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("MarketData")
2024-02-18 19:40:46,731 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-18 19:40:46,736 INFO sqlalchemy.engine.Engine 
CREATE TABLE "MarketData" (
	"index" BIGINT, 
	"Date" DATETIME, 
	"Copper" FLOAT, 
	"Zinc" FLOAT, 
	"RSI_Copper" FLOAT, 
	"MACD_Copper" FLOAT, 
	"MACD_sig_Copper" FLOAT, 
	"MACD_hist_Copper" FLOAT, 
	"RSI_Zinc" FLOAT, 
	"MACD_Zinc" FLOAT, 
	"MACD_sig_Zinc" FLOAT, 
	"MACD_hist_Zinc" FLOAT
)


2024-02-18 19:40:46,737 INFO sqlalchemy.engine.Engine [no key 0.00192s] ()
2024-02-18 19:40:46,741 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_MarketData_index" ON "MarketData" ("index")
2024-02-18 19:40:46,742 INFO sqlalchemy.engine.Engine [no key 0.00118s] ()
